In [13]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
from long_lat import longitude_latitude_dict

In [14]:
# read csv with addresses
df = pd.read_csv('merged_data.csv')

geolocator = Nominatim(user_agent="my_app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1) #need delay 1 sec

for address in df['location']:
    if address in longitude_latitude_dict:
        continue
    try:
        print(f"Geocoding address: {address}")
        location = geocode(address)
        if location:
            longitude_latitude_dict[address] = [location.latitude, location.longitude]

        else:
            longitude_latitude_dict[address] = [None, None]

    except Exception as e:
        print(f"Error geocoding address: {address}")
        print(f"Error: {e}")
    time.sleep(1) 

Geocoding address: 327 North Harbor Boulevard    Los Angeles, CA 90731
Geocoding address: 14715 West Burbank Boulevard    Los Angeles, CA 91367
Geocoding address: 7617 Santa Monica Blvd    West Hollywood, CA 90046
Geocoding address: 2801 Kelvin Avenue    Irvine, CA 92614
Geocoding address: 9530 Reseda Blvd    Los Angeles, CA 91324
Geocoding address: 3642 Motor Ave    Los Angeles, CA 90034
Geocoding address: 8350 Santa Monica Boulevard    West Hollywood, CA 90069
Geocoding address: 11819 Wilshire Boulevard    Los Angeles, CA 90025
Geocoding address: 2345 Santa Fe Avenue    Los Angeles, CA 90058
Geocoding address: 820 N. Central Avenue    Glendale, CA 91203
Geocoding address: 2620 Menlo Avenue    Los Angeles, CA 90007
Geocoding address: 1120 S Olive St    Los Angeles, CA 90015
Geocoding address: 212 Eucalyptus Drive    El Segundo, CA 90245
Geocoding address: 21110 West Oxnard Street    Los Angeles, CA 91367
Geocoding address: 272 W Bellevue Drive    Pasadena, CA 91105
Geocoding address: 

RateLimiter caught an error, retrying (0/2 tries). Called with (*('1932 W Park Avenue    Los Angeles, CA 90026',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 1419, in getresponse
    response.begin()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.1

Geocoding address: 806 North Sweetzer Avenue    Los Angeles, CA 90069
Geocoding address: 12835 Ventura Blvd    Los Angeles, CA 91604
Geocoding address: 16201 San Fernando Mission Boulevard    Los Angeles, CA 91344
Geocoding address: 1437 6th Street    Santa Monica, CA 90401
Geocoding address: 1818 N Cherokee Avenue    Los Angeles, CA 90028
Geocoding address: 2300 Wilshire Blvd    Santa Monica, CA 90403
Geocoding address: 1100 Steely    Irvine, CA 92614
Geocoding address: 206 W 8th St    Los Angeles, CA 90014
Geocoding address: 718 Rose Avenue    Los Angeles, CA 90291
Geocoding address: 1700 West Olympic Boulevard    Los Angeles, CA 90015
Geocoding address: 401 West 1st Street    Santa Ana, CA 92701
Geocoding address: 1122 S Roxbury Dr    Los Angeles, CA 90035
Geocoding address: 3405 West Carson Street    Torrance, CA 90503


In [ ]:
# longitude_latitude_dict to long_lat.py
with open('long_lat.py', 'w') as f:
    f.write(f"longitude_latitude_dict = {longitude_latitude_dict}")


In [ ]:
df = df.copy()
df['latitude'] = df['location'].map(lambda x: longitude_latitude_dict.get(x, [None, None])[0])
df['longitude'] = df['location'].map(lambda x: longitude_latitude_dict.get(x, [None, None])[1])

# remove rows with failed geocoding
df_clean = df.dropna(subset=['latitude', 'longitude'])

# make heatmap
m = folium.Map(location=[34.0522, -118.2437], zoom_start=11)
locations = [[row['latitude'], row['longitude']] for _, row in df_clean.iterrows()]
HeatMap(locations).add_to(m)
m.save('los_angeles_urbanize_heatmap.html')
print(f"Successfully geocoded {len(df_clean)} out of {len(df)} addresses")
print("Heatmap saved as 'los_angeles_urbanize_heatmap.html'")

Successfully geocoded 18 out of 3692 addresses
Heatmap saved as 'los_angeles_development_heatmap.html'
